<a href="https://colab.research.google.com/github/mohsenh17/jaxLearning/blob/main/flax/basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from flax import nnx
import jax
from jax import numpy as jnp


# Simple Linear

In [3]:
class Linear(nnx.Module):

  def __init__(self, din:int, dout:int, *, rngs:nnx.Rngs): # * force the succeeding variables must be provided with keyword arg
    key = rngs.params()
    self.w = nnx.Param(jax.random.uniform(key, (din, dout)))
    self.b = nnx.Param(jnp.zeros((dout,)))
    self.din, self.dout = din, dout

  def __call__(self, x:jnp.ndarray):
    return jnp.dot(x, self.w) + self.b

model = Linear(3, 5, rngs=nnx.Rngs(params=0))
x = jnp.ones((6, 3))
y = model(x)
print(y)
nnx.display(model)


[[1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]
 [1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]
 [1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]
 [1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]
 [1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]
 [1.4492468 1.6527531 1.1650311 1.6205449 1.5738977]]
Linear(
  w=Param(
    value=Array(shape=(3, 5), dtype=float32)
  ),
  b=Param(
    value=Array(shape=(5,), dtype=float32)
  ),
  din=3,
  dout=5
)
